In [1]:
# ======================================================================
# SCRIPT 1 — RÉGRESSION SUPERVISÉE (OZONE)
# ======================================================================
# Objectif :
#   - Préparation données (dummies, scaling…)
#   - Modèles : MCO, LASSO, RIDGE, ELASTIC NET
#   - Validation croisée interne (choix lambda)
#   - Validation croisée externe (prédictions honnêtes)
#   - Table finale PREV
# ======================================================================

# ======================================================================
# 0) IMPORTS
# ======================================================================
import pandas as pd
import numpy as np

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import KFold
from sklearn.linear_model import LinearRegression, LassoCV, RidgeCV, ElasticNetCV, Lasso, Ridge, ElasticNet


In [4]:

# ======================================================================
# 1) LECTURE + PRÉPARATION DES DONNÉES
# ======================================================================

don = pd.read_table("ozone.txt", sep=";")
don = don.rename(columns={"O3": "Y"})

# variables inutiles selon ton workflow
don = don.drop(columns=["Date"])

# ----------------------------------------------------------------------
# 1A — Codage disjonctif complet (pour référence, même si BASE = sans poly)
# ----------------------------------------------------------------------
don_dummies = pd.get_dummies(
    don,
    columns=["nebulosite", "vent"],
    drop_first=False
)

# ----------------------------------------------------------------------
# 1B — Séparation variables
# ----------------------------------------------------------------------
X = don_dummies.drop(columns=["Y"])     # variables explicatives
Y = don_dummies["Y"].to_numpy()         # cible

# ----------------------------------------------------------------------
# 1C — Jeu "BASE" = numériques + dummies
# ----------------------------------------------------------------------
Xbase = X.copy()

# Standardisation futur
scaler = StandardScaler()


In [5]:

# ======================================================================
# 2) CRÉATION DES BLOCS — VALIDATION CROISÉE EXTERNE
# ======================================================================

np.random.seed(1234)
nb = 10
n = don_dummies.shape[0]

tmp = np.arange(n) % nb
bloc = np.random.choice(tmp, size=n, replace=False)

don_dummies["bloc"] = bloc

# Tableau final qui contiendra toutes les prédictions "honnêtes"
PREV = pd.DataFrame({
    "bloc": bloc,
    "Y": Y,
    "MCO": 0.0,
    "lasso": 0.0,
    "ridge": 0.0,
    "elast": 0.0
})


In [6]:

# ======================================================================
# 3) VALIDATION CROISÉE EXTERNE
# ======================================================================

for i in range(nb):
    print(f"\n### Bloc {i} ###")

    # -----------------------------
    # 3.1 — Split TRAIN / TEST
    # -----------------------------
    Xtrain = Xbase[don_dummies["bloc"] != i]
    Xtest  = Xbase[don_dummies["bloc"] == i]

    Ytrain = Y[don_dummies["bloc"] != i]
    Ytest  = Y[don_dummies["bloc"] == i]

    # -----------------------------
    # 3.2 — MODELE 1 : MCO
    # -----------------------------
    reg = LinearRegression()
    reg.fit(Xtrain, Ytrain)
    prev = reg.predict(Xtest)
    PREV.loc[PREV.bloc == i, "MCO"] = prev

    # -----------------------------
    # 3.3 — MODELE 2 : LASSO (VC interne)
    # -----------------------------
    kf = KFold(n_splits=10, shuffle=True, random_state=0)
    lassocv = LassoCV(cv=kf, n_alphas=200)

    # scaling sur le TRAIN uniquement
    scaler_train = StandardScaler().fit(Xtrain)
    Xtrain_cr = scaler_train.transform(Xtrain)
    Xtest_cr  = scaler_train.transform(Xtest)

    lassocv.fit(Xtrain_cr, Ytrain)
    lasso = Lasso(alpha=lassocv.alpha_).fit(Xtrain_cr, Ytrain)
    PREV.loc[PREV.bloc == i, "lasso"] = lasso.predict(Xtest_cr)

    # -----------------------------
    # 3.4 — MODELE 3 : RIDGE (VC interne)
    # -----------------------------
    alphas_ridge = lassocv.alphas_ * 100  # grille large

    ridgecv = RidgeCV(alphas=alphas_ridge, cv=kf)
    ridgecv.fit(Xtrain_cr, Ytrain)

    ridge = Ridge(alpha=ridgecv.alpha_).fit(Xtrain_cr, Ytrain)
    PREV.loc[PREV.bloc == i, "ridge"] = ridge.predict(Xtest_cr)

    # -----------------------------
    # 3.5 — MODELE 4 : ELASTIC NET (VC interne)
    # -----------------------------
    enetcv = ElasticNetCV(cv=kf, l1_ratio=[0.5], n_alphas=200)
    enetcv.fit(Xtrain_cr, Ytrain)

    enet = ElasticNet(alpha=enetcv.alpha_, l1_ratio=0.5).fit(Xtrain_cr, Ytrain)
    PREV.loc[PREV.bloc == i, "elast"] = enet.predict(Xtest_cr)



### Bloc 0 ###

### Bloc 1 ###

### Bloc 2 ###

### Bloc 3 ###

### Bloc 4 ###

### Bloc 5 ###


c:\Users\sikal\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:695: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 8.278e+00, tolerance: 2.517e+00
  model = cd_fast.enet_coordinate_descent(
c:\Users\sikal\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:695: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 8.278e+00, tolerance: 2.517e+00
  model = cd_fast.enet_coordinate_descent(



### Bloc 6 ###

### Bloc 7 ###

### Bloc 8 ###

### Bloc 9 ###


In [ ]:

# ======================================================================
# 4) RÉSULTATS
# ======================================================================

# RMSE par modèle # je peux demander des voir les autres fonction de perte...
def rmse(y, yhat):
    return np.sqrt(np.mean((y - yhat)**2))

tab_perf = pd.DataFrame({
    "RMSE_MCO":   [rmse(PREV["Y"], PREV["MCO"])],
    "RMSE_LASSO": [rmse(PREV["Y"], PREV["lasso"])],
    "RMSE_RIDGE": [rmse(PREV["Y"], PREV["ridge"])],
    "RMSE_ENET":  [rmse(PREV["Y"], PREV["elast"])]
})

print("\n=== PERFORMANCE GLOBALE ===")
print(tab_perf)

# Sauvegarde
PREV.to_csv("PREV_regression_base.csv", index=False)
tab_perf.to_csv("perf_regression_base.csv", index=False)



=== PERFORMANCE GLOBALE ===
    RMSE_MCO  RMSE_LASSO  RMSE_RIDGE  RMSE_ENET
0  15.083262   14.899323   14.527871  14.822131
